In [1]:
from bokeh import plotting as plt
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [2]:
import pathlib
mcnp_dir = pathlib.Path.cwd() / "mcnp"

# Load secrets

In [3]:
from plumbum import cmd

lookup_secret = lambda field: cmd.secret_tool("lookup", "app", "phymat", "field", field).strip()
user = lookup_secret("user")
domain = lookup_secret("domain")
port = lookup_secret("port")
password = lookup_secret("password")

# Start SSHFS

In [4]:
import pexpect
p = pexpect.spawn('bash')
p.sendline(f'sshfs {user}@{domain}:mcnp {mcnp_dir} -p {port}')
p.expect("password:")
p.sendline(password)
p.sendline('exit')
p.expect_exact(pexpect.EOF)

0

# Define SSH session

In [5]:
from plumbum.machines.paramiko_machine import ParamikoMachine
from plumbum.commands import BaseCommand
from proxy import Proxy
    
class RemoteCommandProxy(Proxy):
    proxy_cls = BaseCommand
    
    def __call__(self, *args, _kwargs=None, **kwargs):
        new_args = list(args)
        new_args.extend(f'{k}={v}' for k, v in kwargs.items())
        if _kwargs is None:
            _kwargs = {}
        return object.__getattribute__(self, '_obj')(*new_args, _kwargs)
        
    
rem = ParamikoMachine(domain, user, port, password, keep_alive=30)
rem._cwd = rem.cwd.chdir('mcnp')
remote = RemoteCommandProxy(rem)

In [6]:
from plumbum import ProcessExecutionError
from colorama import Fore
import re

_clean =  remote['./clean.sh']
_mcnp = remote['./mcnp']

replacers = {r'warning\.': Fore.YELLOW, r'fatal error\.': Fore.RED, 
             r'\d+ particles got lost.': Fore.RED}

def highlight(string, replacers):
    pattern = '|'.join(f'({p})' for p in replacers)
    def replacer(m):
        code = next(v for i, v in enumerate(replacers.values()) if m.groups()[i] is not None)
        return f"{code}{m.group(0)}{Fore.RESET}"
    return re.sub(pattern, replacer, string)

def mcnp(*args, **kwargs): 
    return highlight(_mcnp(*args, **kwargs), replacers)

def clean():
    try:
        _clean()
    except ProcessExecutionError:
        pass

# Part 1

In [7]:
%%writefile mcnp/pt1.ip
MESSAGE:

Practical Monte Carlo part 1.
C
C Cells
1 1 7.92 1 -2 3 -4 5 -6 (-7:8:-9:10:-11) $ Walls
2 0 (-1:2:-3:4:-5:6) $ Void
3 2 1.0 7 -8 9 -10 11 -12 $ Water
4 0 7 -8 9 -10 12 -6 $ Air gap (void)

C Surfaces
C Define outer walls
1 PX -5.20
2 PX 5.20
3 PY -10.20
4 PY 10.20
5 PZ 0.0
6 PZ 20.0
C Define XY wall surfaces
7 PX -5.00
8 PX 5.00
9 PY -10.0
10 PY 10.0
C Define Z wall / water surfaces
11 PZ 0.20
12 PZ 19.0

C Cell importance MAP
IMP:N 1 0 1 1         $  s
M1   26000.42c 0.74 24000.42c 0.18 28000.42c 0.08 $ Stainless steel
M2   1001.42c 0.6667  8016.42c 0.3333   $ Pure water
C Tallying energy fluence (nX where X is type (2) and n an ID: {1, 2, ...})
F12:N (1 2)
F22:N (3 4)
C Tally energy bins for all tallies (log 10 space)
E0 1E-9 1E-8 1E-7 1E-6 1E-5 1E-4 1E-3 1E-2 1E-1 1 10
C MC type
MODE N
C Thermal neutron induced fission of 235U at (0,0,20mm)
SDEF POS=0.0 0.0 2.0 ERG=D1
SP1  -3 0.988 2.249
C NUMBER OF PARTICLE HISTORIES TO RUN
NPS  20000
PRDMP 0 0 1 1 0

Overwriting mcnp/pt1.ip


In [11]:
clean()

print(mcnp(inp="pt1.ip", mctal="pt1tal"))

 mcnp     ver=4c2  ld=01/20/01     04/02/18 11:37:02                  
 warning.    1 cell densities are greater than 40 gram/cc.
 imcn   is done
 warning.  neutron energy cutoff is below some cross-section tables.
 warning.     5 cross sections modified by free gas thermal treatment.
 dump    1 on file runtpe     nps =         0    coll =              0
                              ctm =      0.00     nrn =              0
 xact   is done
 dynamic storage =    177270 words,     709080 bytes.      cp0 =  0.00
 run terminated when     20000 particle histories were done.
 warning.  tally  12 tfc bin did not pass  7 of 10 statistical checks.
 warning.     2 of   2 tallies did not pass all 10 statistical checks.
 warning.     1 of   2 tallies were all zeros.
 warning.     1 of   2 tallies had bins with large relative errors.
 dump    2 on file runtpe     nps =     20000    coll =        4631219
                              ctm =      0.10     nrn =       78977929
 tally data written to fi

In [12]:
from mctal import tokenizer
tokens = [*tokenizer.tokenize_file("mcnp/pt1tal")]

In [13]:
from derp import parse
from mctal import g
mctal = next(iter(parse(g.mctal, tokens)))
for tally in mctal.tallies:
    print(tally.data)

((0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (7.63664e-08, 1.0), (0.0, 0.0), (6.28655e-08, 1.0), (1.39232e-07, 0.7104))
((0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0))


# Part 3

In [14]:
%%writefile mcnp/pt3.ip
MESSAGE:

Practical Monte Carlo part 3.
C
C Cells
1 1 7.92 1 -2 3 -4 5 -6 (-7:8:-9:10:-11) $ Walls
4 0 7 -8 9 -10 12 -6 $ Air gap (void)
5 3 2.3 -5 13 1 -2 3 -4$ Concrete floor 
C Define uranium cylinder cells:
6 4 19.2 -14 11 -18
7 4 19.2 -15 11 -18
8 4 19.2 -16 11 -18
9 4 19.2 -17 11 -18
C Boolean cells (depend upon above implicitly)
2 0 6:-13:-1:2:-3:4 $ Main void
3 2 1.0 7 -8 9 -10 11 -12 ((14 15 16 17):18) $ Water

C Surfaces (100.4x100.4x60 bucket (external) including 2mm wall thickness)
C Define outer walls:
1 PX -100.20
2 PX 100.20
3 PY -100.20
4 PY 100.20
5 PZ 0.0
6 PZ 60.0
C Define XY wall surfaces
7 PX -100.00
8 PX 100.00
9 PY -100.0
10 PY 100.0
C Define Z wall / water surfaces
11 PZ 0.20 $ Internal floor
12 PZ 58.00 $ Water surface
13 PZ -304.8 $ Concrete floor lower surface
C Define uranium cylinder surfaces:
14 C/Z -20.0 0.0 7.5
15 C/Z 0.0 20.0 7.5
16 C/Z 20.0 0.0 7.5
17 C/Z 0.0 -20.0 7.5
C Define cylinder ceiling:
18 PZ 25.2

C Cell importance MAP
IMP:N 1 1 1 1 1 1 1 1 1 
M1  26000.42c 0.74 24000.42c 0.18 28000.42c 0.08 $ Stainless steel
M2  1001.42c 0.6667 8016.42c 0.3333   $ Pure water
M3  8016.42c 0.53 14000.42c 0.34 20000.42c 0.10 1001.42c 0.03 $ Concrete
M4  92238.42c 0.8 92235.42c 0.2 $ 80% 238U, 20% 235U
C Tallying energy fluence (nX where X is type (2) and n an ID: {1, 2, ...})
C Tally energy bins for all tallies (log 10 space):
E0 1E-9 1E-8 1E-7 1E-6 1E-5 1E-4 1E-3 1E-2 1E-1 1 10
C MC type
MODE N
C Source definitions:
KCODE 500 0.5 20 1000
KSRC 20 0 12.7 -20 0 12.7 0 20 12.7 0 -20 12.7
C Control output data format
PRDMP 0 0 1 1 0

Overwriting mcnp/pt3.ip


In [ ]:
clean()
print(mcnp(inp="pt3.ip", mctal='pt3tal'))

Lessons learned from MCNP4
* Plotting window doesn't show boundary errors necessarily intuitively 
* KCODE may require specifying upper bound on alloc space (10x n hist here)
* 

In [7]:
tokens = [*tokenizer.tokenize_file("mcnp/mctal")]
mctal = next(iter(parse(g.mctal, tokens)))
for tally in mctal.tallies:
    print(tally.data)

NameError: name 'tokenizer' is not defined

In [340]:
a = (20**2/2)**0.5
r = 7.5
(mcnp_dir/"demo.ip").write_text(
f"""Example 5-3, Hexahedral Lattices. Updated Lattice
C Cell Cards
1 1 9.9270e-2 -1 5 -6 u=1 imp:n=1 $Pu Soln.
2 0 -1 6 -7 u=1 imp:n=1 $ void above
3 2 8.6360e-2 -2 #1 #2 u=1 imp:n=1 $ SS
4 0 2 u=1 imp:n=1 $ void
5 0 -8 9 -10 11 lat=1 fill=1 u=2 imp:n=1 $ lattice
6 0 9 -12 11 -14 3 -4 fill=2 imp:n=1 $ window
7 0 -9:12:-11:14:-3:4 imp:n=0 $ outside

C Solution Cylinder Surface Cards
1 cz {r}
2 cz {r}
5 pz 0.0
6 pz 39.24
7 pz 101.7
C Beginning of lattice surfaces
8 px {a}
9 px {-a}
10 py {a}
11 py {-a}
C Beginning of Window Surfaces
3 pz -1.0
4 pz 102.7
12 px {a*n}
14 py {a*n}

C Control Cards
kcode 5000 1.0 50 250
c one source point in each volume of Pu Solution
ksrc 0 0 19.62 35.58 0 19.62 71.16 0 19.62
     0 35.58 19.62 35.58 35.58 19.62 71.16 35.58 19.62
C Material cards
M1  92238.42c 0.8 92235.42c 0.2 $ 80% 238U, 20% 235U
M2  26000.42c 0.74 24000.42c 0.18 28000.42c 0.08 $ Stainless steel""")

984

In [330]:
(20**2/2)**0.5 * 3

42.42640687119285

In [329]:
clean()

print(mcnp(inp="demo.ip"))

 mcnp     ver=4c2  ld=01/20/01     04/02/18 00:01:03                  
 warning.  universe map (print table 128) disabled.
 
 warning.  surface      9   appears more than once in a chain.
 warning.  surface     11   appears more than once in a chain.
  total fission nubar data are being used.
 imcn   is done
 warning.   92235.42c lacks delayed neutron cross sections.
 warning.   92238.42c lacks delayed neutron cross sections.
 warning.  neutron energy cutoff is below some cross-section tables.
 warning.     5 cross sections modified by free gas thermal treatment.
 dump    1 on file runtpe     nps =         0    coll =              0
                              ctm =      0.00     nrn =              0
 source distribution to file srctp           cycle =     0
 xact   is done
 dynamic storage =    931196 words,    3724784 bytes.      cp0 =  0.01

 cycle    k(col)       ctm     active     k(col)    std dev        fom
     1   1.47200   0.00498
     2   1.40253   0.01108
     3   1.39542

In [ ]:
rad